In [24]:
import scipy.linalg as la
import numpy as np
import random
import matplotlib.pyplot as plt


def erdos_renyi_graph(N, M):
  graph = []
  node_list = [i for i in range(N)]
  numEdges = 0
  while numEdges < M:
    edge = random.sample(node_list, 2)
    if edge not in graph:
      numEdges += 1
      graph.append(edge)
  return graph

def find_degrees(graph_dict, node_list):
  degs = []
  for i in node_list:
    degs.append(len(graph_dict[i]))
  return degs

def barbell(n1, n2):
  graph = []
  for node in range(n1):
    for neighbor in range(n1):
      if (node != neighbor):
        graph.append([node, neighbor])
  for node in range(n1, n1+n2):
    for neighbor in range(n1, n1+n2):
      if (node != neighbor):
        graph.append([node, neighbor])
  node1 = np.random.randint(n1)
  node2 = np.random.randint(n1, n1+n2)
  graph.append([node1, node2])
  graph.append([node2, node1])
  return graph

def stochastic_block(n, r, P):
  graph = []
  node_list = np.arange(n)
  communities = np.array_split(node_list, r)
  for i in range(r):
    for j in range(r):
      C_i = communities[i]
      C_j = communities[j]
      p_ij = P[i][j]
      for node1 in C_i:
        for node2 in C_j:
          if node1 != node2:
            if np.random.rand() < p_ij:
                graph.append([node1, node2])
  return graph

def multi_barbell(N, numBoundaryEdges):
  n1 = int(N/2)
  n2 = int(N/2)
  graph = []
  boundaryNodes1 = random.sample([i for i in range(n1)], numBoundaryEdges)
  boundaryNodes2 = random.sample([i for i in range(n1, N)], numBoundaryEdges)
  for node in range(n1):
    for neighbor in range(n1):
      if (node != neighbor):
        graph.append([node, neighbor])
  for node in range(n1, n1+n2):
    for neighbor in range(n1, n1+n2):
      if (node != neighbor):
        graph.append([node, neighbor])
  for i in range(numBoundaryEdges):
    node1 = random.sample(boundaryNodes1, 1)[0]
    # print (node1)
    node2 = random.sample(boundaryNodes2, 1)[0]
    boundaryNodes1.remove(node1)
    boundaryNodes2.remove(node2)
    graph.append([node1, node2])
    graph.append([node2, node1])
  # print (graph)
  return graph

def edge_density(part1, part2, graph_dict, N):
  E = 0
  for node in part1:
    for neighbor in graph_dict[node]:
      if neighbor in part2:
        E += 1

  return N*(E/(len(part1)*len(part2)))

def sparsest_cut(graph_dict, node_list):
  L = graph_to_laplacian(graph_dict, node_list)
  results = la.eig(L)
  eigvalues = results[0].real
  eigvectors = results[1]
  idx = eigvalues.argsort()[::-1]
  eigvectors = eigvectors[:,idx]  
  second_smallest_vec = eigvectors[:, -2]
  vec = second_smallest_vec.T
  ids = vec.argsort()[::-1]
  curr_density = len(node_list)
  curr_i = 1
  for i in range(int(len(node_list)/4), int((3*len(node_list))/4)):
    part1 = []
    part2 = []
    for id in ids[:i]:
      part1.append(node_list[id])
    for j in ids[i:]:
      part2.append(node_list[j])
    density = edge_density(part1, part2, graph_dict, len(node_list))
    if density < curr_density and density > 0:
      curr_density = density
      curr_i = i
  part1 = ids[:curr_i]
  part2 = ids[curr_i:]
  nodes1 = []
  for i in part1:
    nodes1.append(node_list[i])
  nodes2 = []
  for i in part2:
    nodes2.append(node_list[i])
  return nodes1, nodes2

# def find_boundary_nodes(graph, part1, part2):
#   nodes = []
#   for edge in graph:
#     if (edge[0] in part1 and edge[1] in part2) or (edge[1] in part1 and edge[0] in part2):
#       # print (edge)
#       if edge[0] not in nodes:
#         nodes.append(edge[0])
#       if edge[1] not in nodes:
#         nodes.append(edge[1])
#   # print (nodes)
#   return nodes

def find_boundary_nodes(graph_dict, nodes1, nodes2):
  nodes = []
  for node in graph_dict:
    for neighbor in graph_dict[node]:
      # print (node)
      # print (neighbor)
      if (node in nodes1 and neighbor in nodes2) or (neighbor in nodes1 and node in nodes2):
        if node not in nodes:
          nodes.append(node)
        if neighbor not in nodes:
          nodes.append(neighbor)
    # print (nodes)
  return nodes

def min_cut_antidotes(graph_dict, node_list, initial_recovery_rate, budget, part1, part2, recoveryRates, boundaryThreshold):
  # recoveryRates = {}
  # print ("part1:" + str(part1))
  # print ("part2:" + str(part2))
  # node_list = np.arange(N)
  nodes = find_boundary_nodes(graph_dict, part1, part2)
  antidote_amt = budget/len(nodes)
  if antidote_amt > boundaryThreshold:
    antidote_amt = boundaryThreshold
  for node in node_list:
    if node in nodes:
      # print (node)
      recoveryRates[node] += antidote_amt
      budget -= antidote_amt
  return recoveryRates

def tuples_to_dict(graph, node_list):
  graph_dict = {}
  for i in node_list:
    graph_dict[i] = []
  for edge in graph: 
    graph_dict[edge[0]].append(edge[1])
  return graph_dict

def node_index(node, node_list):
  for index in range(len(node_list)):
    if (node_list[index] == node):
      return index

def graph_to_laplacian(graph_dict, node_list):
  # graph_dict = tuples_to_dict(graph, node_list)
  # print (graph_dict)
  D = find_degrees(graph_dict, node_list)
  L = np.diag(D)
  for node in graph_dict:
    for neighbor in graph_dict[node]:
      L[node_index(node, node_list)][node_index(neighbor, node_list)] = -1
      L[node_index(neighbor, node_list)][node_index(node, node_list)] = -1
  return L

def measureSpectralGap(graph_dict, node_list):
  L = graph_to_laplacian(graph_dict, node_list)
  results = la.eig(L)
  eigvalues = results[0].real

  idx = eigvalues.argsort()[::-1]
  eigvalues = eigvalues[idx]
  # print (eigvalues)
  eig2 = eigvalues[-2]
  return eig2
  
N = 100
r = 2
P = np.zeros((r,r))
P[0][0] = 0.4
P[1][1] = 0.4
P[0][1] = 0.1
P[1][0] = 0.1
graph1 = stochastic_block(N, r, P)
graph2 = multi_barbell(N, 10)
graph3 = erdos_renyi_graph(N, 800)
# print (measureSpectralGap(graph1, np.arange(N)))
# print (measureSpectralGap(graph2, np.arange(N)))
# print (measureSpectralGap(graph3, np.arange(N)))
# edgenums = np.arange(0, 50, 1)
# Y = []
# for num in edgenums:
#   graph = multi_barbell(100, num)
#   spectralgap = measureSpectralGap(graph, np.arange(100))
#   Y.append(spectralgap)
# plt.plot(edgenums, Y)

def degreeProportional(graph_dict, recovery_rates, budget):
    degDict = {}
    sum = 0
    for node in graph_dict:
        degDict[node] = len(graph_dict[node])
        sum += len(graph_dict[node])
    for node in degDict:
        recovery_rates[node] += (degDict[node]/sum)*budget
      
def findNewGraph(graph_dict, nodes1, nodes2):
  new_graph_dict1 = {}
  len1 = 0
  new_graph_dict2 = {}
  len2 = 0
  for node in graph_dict:
    for neighbor in graph_dict[node]:
      if node in nodes1 and neighbor in nodes1:
        if node not in new_graph_dict1:
          new_graph_dict1[node] = []
        new_graph_dict1[node].append(neighbor)
        len1 += 1
      if node in nodes2 and neighbor in nodes2:
        if node not in new_graph_dict2:
          new_graph_dict2[node] = []
        new_graph_dict2[node].append(neighbor)
        len2 += 1
  return new_graph_dict1, len1, new_graph_dict2, len2
    
def newStrategy(graph_dict, recovery_rates, budget, gap_threshold, boundary_threshold, node_list):
  gap = measureSpectralGap(graph_dict, node_list)
  print (gap)
  if (gap < gap_threshold):
    # print (gap)
    nodes1, nodes2 = sparsest_cut(graph_dict, node_list)
    graph_dict1, len1, graph_dict2, len2 = findNewGraph(graph_dict, nodes1, nodes2)
    min_cut_antidotes(graph_dict, node_list, initial_recovery_rate, budget, nodes1, nodes2, recovery_rates, boundary_threshold)
    if (budget > 0):
      budget1 = budget * (len1 / (len1 + len2))
      budget2 = budget * (len2 / (len1 + len2))
      newStrategy(graph_dict1, recovery_rates, budget1, gap_threshold, boundary_threshold, nodes1)
      newStrategy(graph_dict2, recovery_rates, budget2, gap_threshold, boundary_threshold, nodes2)
  else:
    print (recovery_rates)
    degreeProportional(graph_dict, recovery_rates, budget) 
  return recovery_rates

N = 100
node_list = np.arange(N)
recovery_rates = {}
initial_recovery_rate = 0.01
budget = 400
for node in node_list:
  recovery_rates[node] = initial_recovery_rate
newStrategy(tuples_to_dict(graph1, node_list), recovery_rates, budget, 2, 1, node_list)

1.6081893123662134
9.745862714528617
{0: 1.01, 1: 1.01, 2: 1.01, 3: 1.01, 4: 1.01, 5: 1.01, 6: 1.01, 7: 1.01, 8: 1.01, 9: 1.01, 10: 1.01, 11: 1.01, 12: 1.01, 13: 1.01, 14: 1.01, 15: 1.01, 16: 1.01, 17: 1.01, 18: 1.01, 19: 1.01, 20: 1.01, 21: 1.01, 22: 1.01, 23: 1.01, 24: 1.01, 25: 1.01, 26: 1.01, 27: 1.01, 28: 1.01, 29: 1.01, 30: 1.01, 31: 1.01, 32: 1.01, 33: 1.01, 34: 1.01, 35: 1.01, 36: 1.01, 37: 1.01, 38: 1.01, 39: 1.01, 40: 1.01, 41: 1.01, 42: 1.01, 43: 1.01, 44: 1.01, 45: 1.01, 46: 1.01, 47: 1.01, 48: 1.01, 49: 1.01, 50: 1.01, 51: 1.01, 52: 1.01, 53: 1.01, 54: 1.01, 55: 1.01, 56: 1.01, 57: 1.01, 58: 1.01, 59: 1.01, 60: 1.01, 61: 1.01, 62: 1.01, 63: 1.01, 64: 1.01, 65: 1.01, 66: 1.01, 67: 1.01, 68: 1.01, 69: 1.01, 70: 1.01, 71: 1.01, 72: 1.01, 73: 1.01, 74: 1.01, 75: 1.01, 76: 1.01, 77: 1.01, 78: 1.01, 79: 1.01, 80: 1.01, 81: 1.01, 82: 1.01, 83: 1.01, 84: 1.01, 85: 1.01, 86: 1.01, 87: 1.01, 88: 1.01, 89: 1.01, 90: 1.01, 91: 1.01, 92: 1.01, 93: 1.01, 94: 1.01, 95: 1.01, 96: 1.01, 97

{0: 5.072976130015236,
 1: 5.072976130015236,
 2: 4.666678517013712,
 3: 5.682422549517521,
 4: 6.088720162519044,
 5: 4.666678517013712,
 6: 5.072976130015236,
 7: 4.260380904012188,
 8: 4.666678517013712,
 9: 4.057232097511426,
 10: 4.057232097511426,
 11: 4.666678517013712,
 12: 4.869827323514474,
 13: 7.307613001523615,
 14: 4.260380904012188,
 15: 4.869827323514474,
 16: 5.479273743016759,
 17: 4.666678517013712,
 18: 5.479273743016759,
 19: 6.495017775520568,
 20: 5.072976130015236,
 21: 5.2761249365159975,
 22: 5.072976130015236,
 23: 5.682422549517521,
 24: 4.666678517013712,
 25: 5.2761249365159975,
 26: 4.666678517013712,
 27: 5.479273743016759,
 28: 6.088720162519044,
 29: 5.682422549517521,
 30: 5.072976130015236,
 31: 6.698166582021329,
 32: 5.479273743016759,
 33: 4.260380904012188,
 34: 6.088720162519044,
 35: 5.479273743016759,
 36: 5.682422549517521,
 37: 4.666678517013712,
 38: 5.682422549517521,
 39: 5.2761249365159975,
 40: 4.46352971051295,
 41: 4.666678517013712,


TODO:


*   Code up measureSpectralGap()
*   Do experimenting to decide the spectral gap threshold

*   Do experimenting to decide the boundery budget





